In [1]:
import time
import csv
import pandas as pd
import argparse
import numpy as np
import random
from multiprocessing import *


from lib.OsrmEngine import *
from lib.OsrmUtils import *

In [2]:
ghost = '0.0.0.0'
gport = 5000
exe_loc = './osrm-backend-5.6.0/build/osrm-routed'
map_loc = './osrm-backend-5.6.0/greater-london-latest.osrm'

osrm = OsrmEngine(exe_loc, map_loc)
osrm.start_server()

'The routing server "http://0.0.0.0:5000" starts running'

In [7]:
olat = 0.089653
olon = 51.373206
dlat = 0.089282
dlon = 51.350675

stime = time.time()
for i in range(1000):
    o1 = random.uniform(-0.01, 0.01)
    o2 = random.uniform(-0.01, 0.01)
    d1 = random.uniform(-0.01, 0.01)
    d2 = random.uniform(-0.01, 0.01)
    url = create_url(ghost, gport, olat+o1, olon+o2, dlat+d1, dlon+d2)
    out = call_url(url)
etime = time.time()
print(etime-stime)

3.190641164779663


In [8]:
osrm.kill_server()